In [2]:
# CLEAN WORKSPACE

%reset

In [3]:
# Horizon ephemerides converter - functions

import pandas as pd
import re
import random
import numpy as np
from astropy.coordinates import GCRS, ITRS, CartesianRepresentation, CartesianDifferential, SkyCoord
from astropy import units as u
from astropy.time import Time

# function to read and convert results into table ['Calendar_Date', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'ID']
# remember to remove header, footer, 'A.D. ' and ' TDB' from Calendar_Date
# change month in number (es Feb->02)
# input quantities: [position: km, vel: km/s]

def parse_ephemeris(file_path, PertMaxVal=0):

    # Function to read ephimerides from horizon_results and transalte it into the format 
    # ['utc_time', 'pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'navigation_state']
    # file_path: path to the horizon results file
    # PertMaxVal: maximum value in % for the random perturbation to be added to the navigation state (0 means no perturbation)

    data = []
    
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    # Iterate through lines in chunks of 3
    for i in range(0, len(lines), 3):
        # Line 1: Julian Date and Calendar Date
        # Split by '=' and take parts
        time_parts = lines[i].split('=')
#        jd = time_parts[0].strip()              # Julian Date
        cal_date = pd.to_datetime(time_parts[1].strip())
#        cal_date = cal_date.str.replace('A.D. ', '', regex=False)
#        cal_date = cal_date.str.replace(' TDB', '', regex=False).str.strip()

        # Lines 2 & 3: Coordinates and Velocities
        # We use regex to find numbers following labels like X = or VY=
        coords_text = lines[i+1] + " " + lines[i+2]
        # This regex finds all scientific notation numbers
        values = re.findall(r"[-+]?\d*\.\d+[eE][-+]?\d+", coords_text)
        
        # Combine into a single row
        row = [cal_date] + values + [random.choice([0, 3, 3, 3])]
        data.append(row)

    # Define columns
    columns = ['utc_time', 'pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'navigation_state']
    df = pd.DataFrame(data, columns=columns)
    
    # Convert numeric columns to float
    num_cols = ['pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'navigation_state']
    df[num_cols] = df[num_cols].apply(pd.to_numeric)

    if PertMaxVal > 0:
        PertMaxVal = PertMaxVal/100
        df['pos_x'] = df['pos_x'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m
        df['pos_y'] = df['pos_y'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m
        df['pos_z'] = df['pos_z'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m

        df['vel_x'] = df['vel_x'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m / u.s
        df['vel_y'] = df['vel_y'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m / u.s
        df['vel_z'] = df['vel_z'] * (1 + (random.uniform(-PertMaxVal, PertMaxVal))) * 1000 * u.m / u.s
        

    else: 

        df['pos_x'] = df['pos_x'] *1000 * u.m
        df['pos_y'] = df['pos_y'] *1000 * u.m
        df['pos_z'] = df['pos_z'] *1000 * u.m

        df['vel_x'] = df['vel_x'] *1000 * u.m / u.s
        df['vel_y'] = df['vel_y'] *1000 * u.m / u.s
        df['vel_z'] = df['vel_z'] *1000 * u.m / u.s



    
    return df



# PertMaxVal =
# random.randint(1, PertMaxVal)


def rotate_gps_format(data):
    # Change coulumns name according to rotate_gps "I2E" method
    # data is a DataFrame with columns ['utc_time', 'pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'navigation_state']
    # data_ecef is a DataFrame with columns ['randv_mks_0', 'randv_mks_1', 'randv_mks_2', 'randv_mks_3', 'randv_mks_4', 'randv_mks_5', 'navigation_state']
    # the first column is set as the rows index and is the time in datetime format

    data_eci2 = data.copy()
    id = data_eci2.iloc[:,7]
    data_eci2.rename(columns={
        'utc_time': '',
        'pos_x': 'randv_mks_0',
        'pos_y': 'randv_mks_1',
        'pos_z': 'randv_mks_2',
        'vel_x': 'randv_mks_3',
        'vel_y': 'randv_mks_4',
        'vel_z': 'randv_mks_5',
        'navigation_state': 'navigation_state'
    }, inplace=True)
    
    data_eci2.iloc[:, 0] = pd.to_datetime(data_eci2.iloc[:, 0])
    data_eci2 = data_eci2.set_index(data_eci2.columns[0])

    
    return data_eci2, id

def run_Orekit_fit_format(data, id=None):
    # Convert data to the format expected by Orekit fit
    # data is a DataFrame with columns ['', 'randv_mks_0', 'randv_mks_1', 'randv_mks_2', 'randv_mks_3', 'randv_mks_4', 'randv_mks_5', 'navigation_state']
    # The output should be a DataFrame with columns ['utc_time', 'pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'navigation_state']
    
    data_fit = data.copy()
    data_fit = data_fit.reset_index()
    data_fit.rename(columns={
        '': 'utc_time',
        'randv_mks_0': 'ecef_pos_x',
        'randv_mks_1': 'ecef_pos_y',
        'randv_mks_2': 'ecef_pos_z',
        'randv_mks_3': 'ecef_vel_x',
        'randv_mks_4': 'ecef_vel_y',
        'randv_mks_5': 'ecef_vel_z',
        'navigation_state': 'navigation_state'
    }, inplace=True)

    if id is not None:
        data_fit['navigation_state'] = id
    else: 
        data_fit['navigation_state'] = 3
    return data_fit

In [4]:
# converter call
from orbitfit.rotate import rotate_gps

# data_eci = pd.read_csv(r"C:\Users\samuele.barin\Documents\GitHub\orbitfit\output\df_out_interpolated.csv")
data_eci1 = parse_ephemeris(r"C:\Users\samuele.barin\Documents\horizon\horizons_results.txt")
data_eci1.to_pickle(r"C:\Users\samuele.barin\Documents\horizon\horizons_results_eci.pkl")
data_eci, id = rotate_gps_format(data_eci1)
data_ecef = rotate_gps(data_eci, method="I2E")
data_ecef = run_Orekit_fit_format(data_ecef, id)
#data_ecef2.to_csv(r"C:\Users\samuele.barin\Documents\horizon\horizons_results_ecef.csv")
data_ecef.to_pickle(r"C:\Users\samuele.barin\Documents\horizon\horizons_results_ecef.pkl")
